In [ ]:
import os
import random

import torch
from PIL import Image
from diffusers import ZImagePipeline
from nunchaku import NunchakuZImageTransformer2DModel
from tqdm.notebook import tqdm

In [ ]:
prompts = [
    "Children playing in a park",
    "Shoppers walking through wide supermarket aisles",
    "People standing on a train station platform",
    "Commuters waiting at a city bus stop",
    "Pedestrians walking on a wide city sidewalk",
    "Visitors walking through a museum gallery",
    "Students walking in a university corridor",
    "Employees walking in a modern office lobby",
    "Passengers walking in a subway station",
    "Shoppers walking through a wise supermarket aisle",
    "Pedestrians crossing an urban crosswalk",
    "People jogging in a city park",
    "Customers entering a shopping mall",
    "People standing in line at a pharmacy",
    "People walking through neighborhood",
]

In [ ]:
device = 'cuda'
dtype = torch.bfloat16
print(f"Using device: {device}, dtype: {dtype}")
data_dir = '../data/backgrounds'
if os.path.exists(data_dir):
    candidate = data_dir
    i = 0
    while os.path.exists(candidate):
        i += 1
        candidate = f"{data_dir}_{i}"
    data_dir = candidate

os.makedirs(data_dir, exist_ok=False)

In [ ]:
def load_zit_pipeline() -> ZImagePipeline:
    """
Loads the Z-Image Turbo pipeline with Nunchaku INT4 quantised transformer.
    :return: Z-Image Turbo pipeline
    :rtype: ZImagePipeline
    """
    rank = 256
    transformer = NunchakuZImageTransformer2DModel.from_pretrained(
        f"nunchaku-tech/nunchaku-z-image-turbo/svdq-int4_r{rank}-z-image-turbo.safetensors")  # Nunchaku INT4 quantised Z-Image Turbo transformer for memory efficiency

    transformer.set_attention_backend('sage')  # Use sage-attention for better performance

    pipe = ZImagePipeline.from_pretrained("Tongyi-MAI/Z-Image-Turbo", transformer=transformer, torch_dtype=dtype)
    pipe.enable_model_cpu_offload()

    return pipe


def zit_generate(pipe, prompt: str, width: int = 1248, height: int = 832, seed: int | None = None) -> Image.Image:
    """
Generates an image using Z-Image Turbo pipeline.
    :param pipe: Z-Image Turbo pipeline
    :type pipe: ZImagePipeline
    :param prompt: Positive prompt
    :type prompt: str
    :param width: Image width
    :type width: int
    :param height: Image height
    :type height: int
    :param seed: Fixed seed for reproducibility
    :type seed: int | None
    :return: Generated image
    :rtype: Image.Image
    """
    if not seed:
        seed = random.randint(0, 2 ** 31 - 1)
    generator = torch.Generator(device=device).manual_seed(seed)
    img = pipe(
        prompt=prompt,
        num_inference_steps=9, guidance_scale=0.0,
        generator=generator, width=width, height=height
    ).images[0]

    return img

In [5]:
# Load pipeline
torch.cuda.empty_cache()
pipeline = load_zit_pipeline()

2026-02-02 22:43:29,980 - INFO - HTTP Request: HEAD https://huggingface.co/nunchaku-tech/nunchaku-z-image-turbo/resolve/main/svdq-int4_r256-z-image-turbo.safetensors "HTTP/1.1 307 Temporary Redirect"
2026-02-02 22:43:30,136 - INFO - HTTP Request: HEAD https://huggingface.co/nunchaku-ai/nunchaku-z-image-turbo/resolve/main/svdq-int4_r256-z-image-turbo.safetensors "HTTP/1.1 302 Found"


quantization_config: {'method': 'svdquant', 'weight': {'dtype': 'int4', 'scale_dtype': None, 'group_size': 64}, 'activation': {'dtype': 'int4', 'scale_dtype': None, 'group_size': 64}, 'rank': 256, 'skip_refiners': False}, rank=256, skip_refiners=False


Attention backends are an experimental feature and the API may be subject to change.
2026-02-02 22:43:39,854 - INFO - HTTP Request: GET https://huggingface.co/api/models/Tongyi-MAI/Z-Image-Turbo "HTTP/1.1 200 OK"
2026-02-02 22:43:40,012 - INFO - HTTP Request: HEAD https://huggingface.co/Tongyi-MAI/Z-Image-Turbo/resolve/main/model_index.json "HTTP/1.1 307 Temporary Redirect"
2026-02-02 22:43:40,019 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/Tongyi-MAI/Z-Image-Turbo/f332072aa78be7aecdf3ee76d5c247082da564a6/model_index.json "HTTP/1.1 200 OK"


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

In [6]:
img_per_prompt = 25  # Number of images to generate per prompt

total = len(prompts) * img_per_prompt
counter = 0

with tqdm(total=total, position=1, leave=True) as pbar:
    for _ in range(img_per_prompt):
        for p in prompts:
            img = zit_generate(pipeline, p)
            img.save(f'{data_dir}/{counter}.png')
            counter = counter + 1
            pbar.update(1)

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]